---
title: PA 7.1
author: Marvin (Wenxiang) Li
format:
    html:
        toc: true
        code-fold: true
embed-resources: true        
---

In [3]:
import numpy as np
import pandas as pd
Ames = pd.read_csv("https://www.dropbox.com/scl/fi/g0n5le5p6fr136ggetfsf/AmesHousing.csv?rlkey=jlr9xtz1o6u5rghfo29a5c02f&dl=1")
Ames

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


In [4]:
# Get rid of columns with mostly NaN values
good_cols = Ames.isna().sum() < 100
Ames = Ames.loc[:,good_cols]

# Drop other NAs
Ames = Ames.dropna()

In [43]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [12]:
X = Ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = Ames["SalePrice"]

In [44]:
def mse_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    mse = mean_squared_error(y, y_pred)
    return mse

### 1. model 1

In [71]:
numeric_features = ["Gr Liv Area", "TotRms AbvGrd"]
categorical_features = []

ct1 = ColumnTransformer(
    [
     ('scaler', StandardScaler(),numeric_features),
     ('onehot', OneHotEncoder(handle_unknown='ignore'),categorical_features)
], remainder = "drop"
)

model_1 = Pipeline(
  [("preprocessing", ct1),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")


score1 = cross_val_score(model_1, X, y, cv=5, scoring= 'neg_mean_squared_error')
score1 = score1.mean()
score1

-3104032846.4845915

### 2. model 2

In [72]:
numeric_features = ["Gr Liv Area", "TotRms AbvGrd"]
categorical_features = ["Bldg Type"]

ct2 = ColumnTransformer(
    [
     ("standardize", StandardScaler(), numeric_features),
     ('onehot', OneHotEncoder(drop="first",sparse_output = False,handle_unknown='ignore'),categorical_features)
    ], remainder = "drop"
)

model_2 = Pipeline(
  [
    ("column_transformer", ct2),
    ("linear_regression", LinearRegression())
  ]
)
score2 = cross_val_score(model_2, X, y, cv=5, scoring= 'neg_mean_squared_error')
score2 =score2.mean()
score2

-2947908991.3356295

### 3. model 3

In [54]:
ct3 = ColumnTransformer(
    [
     ("standardize", StandardScaler(), ["Gr Liv Area"]),
     ('onehot', OneHotEncoder(drop="first",sparse_output = False,handle_unknown='ignore'),["Bldg Type"])
    ], remainder = "drop"
).set_output(transform="pandas")
ct3.fit_transform(X)[:1]

,standardize__Gr Liv Area,onehot__Bldg Type_2fmCon,onehot__Bldg Type_Duplex,onehot__Bldg Type_Twnhs,onehot__Bldg Type_TwnhsE
0,0.302978,0.0,0.0,0.0,0.0


In [55]:
ct3_int = ColumnTransformer(
    [
        ('interaction1', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'onehot__Bldg Type_2fmCon']),
        ('interaction2', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'onehot__Bldg Type_Duplex']),
        ('interaction3', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'onehot__Bldg Type_Twnhs']),
        ('interaction4', PolynomialFeatures(interaction_only=True, include_bias=False), ['standardize__Gr Liv Area', 'onehot__Bldg Type_TwnhsE'])
    ],
    remainder='drop'
).set_output(transform="pandas")
ct3_int.fit_transform(ct3.fit_transform(X))[:1]

,interaction1__standardize__Gr Liv Area,interaction1__onehot__Bldg Type_2fmCon,interaction1__standardize__Gr Liv Area onehot__Bldg Type_2fmCon,interaction2__standardize__Gr Liv Area,interaction2__onehot__Bldg Type_Duplex,interaction2__standardize__Gr Liv Area onehot__Bldg Type_Duplex,interaction3__standardize__Gr Liv Area,interaction3__onehot__Bldg Type_Twnhs,interaction3__standardize__Gr Liv Area onehot__Bldg Type_Twnhs,interaction4__standardize__Gr Liv Area,interaction4__onehot__Bldg Type_TwnhsE,interaction4__standardize__Gr Liv Area onehot__Bldg Type_TwnhsE
0,0.302978,0.0,0.0,0.302978,0.0,0.0,0.302978,0.0,0.0,0.302978,0.0,0.0


In [73]:
model_3 = Pipeline(
  [
    ("column_transformer", ct3),
    ("interaction", ct3_int),
    ("linear_regression", LinearRegression())
  ]
)
score3 = cross_val_score(model_3, X, y, cv=5, scoring= 'neg_mean_squared_error')
score3 = score3.mean()
score3

-2864871963.27544

### 4. model 4

In [57]:
ct4 = ColumnTransformer(
    [
     ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
     ('onehot', OneHotEncoder(drop="first",sparse_output = False,handle_unknown='ignore'),["Bldg Type"])
    ], remainder = "drop"
).set_output(transform="pandas")
ct4.fit_transform(X)[:1]

,standardize__Gr Liv Area,standardize__TotRms AbvGrd,onehot__Bldg Type_2fmCon,onehot__Bldg Type_Duplex,onehot__Bldg Type_Twnhs,onehot__Bldg Type_TwnhsE
0,0.302978,0.356458,0.0,0.0,0.0,0.0


In [59]:
ct4_poly = ColumnTransformer(
  [
    ("polynomial1", PolynomialFeatures(degree = 5, include_bias=False), ["standardize__Gr Liv Area"]),
    ("polynomial2", PolynomialFeatures(degree = 5, include_bias=False), ["standardize__TotRms AbvGrd"])
  ],
  remainder = "passthrough"
).set_output(transform="pandas")
ct4_poly.fit_transform(ct4.fit_transform(X))[:1]

,polynomial1__standardize__Gr Liv Area,polynomial1__standardize__Gr Liv Area^2,polynomial1__standardize__Gr Liv Area^3,polynomial1__standardize__Gr Liv Area^4,polynomial1__standardize__Gr Liv Area^5,polynomial2__standardize__TotRms AbvGrd,polynomial2__standardize__TotRms AbvGrd^2,polynomial2__standardize__TotRms AbvGrd^3,polynomial2__standardize__TotRms AbvGrd^4,polynomial2__standardize__TotRms AbvGrd^5,remainder__onehot__Bldg Type_2fmCon,remainder__onehot__Bldg Type_Duplex,remainder__onehot__Bldg Type_Twnhs,remainder__onehot__Bldg Type_TwnhsE
0,0.302978,0.091796,0.027812,0.008426,0.002553,0.356458,0.127062,0.045292,0.016145,0.005755,0.0,0.0,0.0,0.0


In [74]:
model_4 = Pipeline(
  [
    ("column_transformer", ct4),
    ("interaction", ct4_poly),
    ("linear_regression", LinearRegression())
  ]
)
score4 = cross_val_score(model_4, X, y, cv=5, scoring= 'neg_mean_squared_error')
score4 = score4.mean()
score4

-3023064058.5582542

### 5. Final results

In [75]:
score = np.array([score1, score2, score3, score4])
score = score * -1
score = pd.DataFrame(score, index = ["model 1", "model 2", "model 3", "model 4"], columns = ["MSE"])
score

,MSE
model 1,3.104033e+09
model 2,2.947909e+09
model 3,2.864872e+09
model 4,3.023064e+09


- Model 3 fits the best in this case

-